In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join
import matplotlib.pyplot as plt
pathIn= '/kaggle/input/eveerybody-dance-nowsubject-2/subject2/train/train_img/'
pathOut = 'videosourcesubject2.avi'
fps = 20
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
#files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#files.sort(key = lambda x: x[5:-4])
for i in range(1000):
    filename=pathIn + files[i]
    if(i==50):
        print(files[i])
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()

In [ ]:
import matplotlib.pyplot as plt
import cv2
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import keras
import numpy as np


In [ ]:

class My_Custom_Generator(keras.utils.Sequence) :
    def __init__(self, image_filenames_label,image_filenames_img, labels, batch_size) :
        self.image_filenames = [image_filenames_label,image_filenames_img]
        self.labels = labels
        self.batch_size = batch_size
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
    def __getitem__(self, idx) :
        batch_x = [self.image_filenames[0][idx * self.batch_size : (idx+1) * self.batch_size],self.image_filenames[1][idx * self.batch_size : (idx+1) * self.batch_size]]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
       
    
        return (batch_x), np.array(batch_y)

In [ ]:
class My_Custom_Generator_gan(keras.utils.Sequence) :
    def __init__(self, image_filenames_label, labels,images_filenames_img, batch_size) :
        self.image_filenames = image_filenames_label
        self.labels = [labels,images_filenames_img]
        self.batch_size = batch_size
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = [self.labels[0][idx * self.batch_size : (idx+1) * self.batch_size],self.labels[1][idx * self.batch_size : (idx+1) * self.batch_size]]
       
    
        return np.array(batch_x),(batch_y)

In [ ]:
def load_images(path1,path2, size=(256,512)):
    src_list,tar_list = list(), list()
    count1=0
    count2=0
    for filename in listdir(path1):
        if(count1==150):
            break
        pixels1 = load_img(path1 + filename, target_size=size)
        pixels1 = img_to_array(pixels1)
        src_list.append(pixels1)
        count1+=1

    for filename in listdir(path2):
        if(count2==150):
            break
        pixels2 = load_img(path2 + filename, target_size=size)
        pixels2 = img_to_array(pixels2)
        tar_list.append(pixels2)
        count2+=1
    return [asarray(src_list), asarray(tar_list)]


path1 ='/kaggle/input/everybody-dance-now1/subject4/train/train_label/'
path2='/kaggle/input/everybody-dance-now1/subject4/train/train_img/'
[src_images, tar_images] = load_images(path1,path2)
print('Loaded: ', src_images.shape, tar_images.shape)
filename = 'dancex_256.npz'
savez_compressed(filename, a=src_images, b=tar_images)
print('Saved dataset: ', filename)

In [ ]:
def load_real_samples(filename):
    data = load(filename)
    X1=data['a']
    X2=data['b']
    X1 = (X1-127.5)/127.5
    X2 = (X2-127.5)/127.5
    return [X1, X2]
from numpy import load
dataset = load_real_samples('dancex_256.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]

In [ ]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot
import cv2


def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape)
    in_target_image = Input(shape=image_shape)
    merged = Concatenate()([in_src_image, in_target_image])
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    model = Model([in_src_image, in_target_image], patch_out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, skip_in])
    g = Activation('relu')(g)
    return g

def define_generator(image_shape=(256,512,3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    model = Model(in_image, out_image)
    return model

def define_gan(g_model, d_model, image_shape):
    d_model.trainable = False
    in_src = Input(shape=image_shape)
    gen_out = g_model(in_src)
    dis_out = d_model([in_src, gen_out])
    model = Model(in_src, [dis_out, gen_out])
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

def load_real_samples(filename):
    data = load(filename)
    X1, X2 = data['arr_0'], data['arr_1']
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

def generate_real_samples(dataset, n_samples, patch_shape):
    trainA, trainB = dataset
    ix = randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[ix], trainB[ix]
    y = ones((n_samples, patch_shape, patch_shape*2, 1))
    return [X1, X2], y

def generate_fake_samples(g_model, samples, patch_shape):
    X = g_model.predict(samples)
    y = zeros((len(X), patch_shape, patch_shape*2, 1))
    return X, y

def summarize_performance(step, g_model, dataset, n_samples=1):
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
 
    print("Pose stick figure")
    for i in range(n_samples):
        pyplot.figure(figsize=(6.5,6.5))
#         pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        v=pyplot.imshow(X_realA[i])
        
        pyplot.show()
   
    print("Generated fake image")
    for i in range(n_samples):
        pyplot.figure(figsize=(6.5,6.5))
        #pyplot.subplot(1, 2, 1)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
       # print(X_fakeB[i].shape)
        pyplot.show()

    print("Real Image")
    for i in range(n_samples):
        pyplot.figure(figsize=(6.5,6.5))
        #pyplot.subplot(1,2,2)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
        pyplot.show()
    #filename1 = 'plot_%06d.png' % (step+1)
    #pyplot.savefig(filename1)
    #pyplot.close()
    #filename2 = 'model_%06d.h5' % (step+1)
    #g_model.save(filename2)
    #print('Saved: %s and %s' % (filename1, filename2))


def train(d_model, g_model, gan_model, dataset, n_batch=150):
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    
    n_steps=20
    
    for i in range(n_steps): 
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        X_realA_train=X_realA[:120]
        X_realA_val=X_realA[120:]
        X_realB_train=X_realB[:120]
        X_realB_val=X_realB[120:]
        X_fakeB_train=X_fakeB[:120]
        X_fakeB_val=X_fakeB[120:]
        y_real_train=y_real[:120]
        y_real_val=y_real[120:]
        y_fake_train=y_fake[:120]
        y_fake_val=y_fake[120:]
        d_model.trainable=True
        batch_size=15
        my_dis_gen_real=My_Custom_Generator(X_realA_train,X_realB_train,y_real_train,batch_size)
        my_val_gen_real=My_Custom_Generator(X_realA_val,X_realB_val,y_real_val,batch_size)
        d_model.fit_generator(generator=my_dis_gen_real,
                   steps_per_epoch = int(150// batch_size),
                   epochs = 100,verbose = 0,validation_data = my_val_gen_real)
        # d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        my_dis_gen_fake=My_Custom_Generator(X_realA_train,X_fakeB_train,y_fake_train,batch_size)
        my_val_gen_fake=My_Custom_Generator(X_realA_val,X_fakeB_val,y_fake_val,batch_size)
        d_model.fit_generator(generator=my_dis_gen_fake,
                   steps_per_epoch = int(150// batch_size),
                   epochs = 100, verbose = 0,validation_data = my_val_gen_fake)
        # d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        d_model.trainable=False
        #g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        my_gan_gen=My_Custom_Generator_gan(X_realA_train,y_real_train,X_realB_train,batch_size)
        my_val_gen=My_Custom_Generator_gan(X_realA_val,y_real_val,X_realB_val,batch_size)
        gan_model.fit_generator(generator=my_gan_gen,
                   steps_per_epoch = int(150// batch_size),
                   epochs = 100, verbose = 0, validation_data = my_val_gen)
        print('Epoch %d/%d completed' % (i+1,n_steps))
        if (i) % (2) == 0:
            summarize_performance(i, g_model, dataset)
        g_model_json = g_model.to_json()
        with open("g_model.json", "w") as json_file:
            json_file.write(g_model_json)
        g_model.save_weights("modelgen.h5")
        print("Saved model to disk")


d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)
train(d_model, g_model, gan_model, dataset)

In [ ]:
import matplotlib.pyplot as plt
import cv2
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import keras
import numpy as np

In [ ]:
from keras.models import model_from_json
json_file = open('/kaggle/input/final00000/g_model.json', 'r')
loadedx_model_json = json_file.read()
json_file.close()
loadedx_model = model_from_json(loadedx_model_json)
loadedx_model.load_weights("/kaggle/input/final00000/modelgen.h5")
print("Loaded model from disk")
 

In [ ]:
plt.figure()
plt.imshow(src_images[6].reshape(256,512,3)/255)
plt.show()
plt.figure()
plt.imshow(tar_images[6].reshape(256,512,3)/255)
plt.show()




In [ ]:
#x=x.reshape(1,256,512,3)
x=(x-127.5)/127.5
x=loadedx_model.predict(src_images)
x = (x + 1) / 2.0
print(x.shape)
plt.imshow(x.reshape(256,512,3))

In [ ]:
import os
import numpy as np
#from PIL import Image
count1=0
size=(256,512)
#path1="/kaggle/input/everybody-dance-nowsubject-1/subject1/train/train_label/"
#pixels1 = load_img(path1 , target_size=size)
#x = img_to_array(pixels1)
if not os.path.isdir("output_images_subject2"):
    os.mkdir("output_images_subject2")
path1="/kaggle/input/eveerybody-dance-nowsubject-2/subject2/train/train_label/"
i=0
for filename in listdir(path1):
        if(count1==1000):
            break
        count1=count1+1
        pixels1 = load_img(path1 + filename, target_size=size)
        x = img_to_array(pixels1)
        x=(x-127.5)/127.5
        x=x.reshape(1,256,512,3)
        pixels1=loadedx_model.predict(x)
        pixels1 = (pixels1 + 1) / 2.0
        print(pixels1.shape)
        pixels1=pixels1.reshape(256,512,3)
        plt.figure()
        plt.axis("off")
        plt.imshow(pixels1)
        plt.savefig("output_images_subject2/final_{0}.png".format(i+1))
        plt.show()
#         plt.savefig("output_images/{:06d}.format(i+1).png",pixels1)
        #plt.imshow("output_images")
        i=i+1
       
        
        
        
#x=plt.imread("/kaggle/input/eveerybody-dance-nowsubject-2/subject2/train/train_label/frame003000.png")
#print(x.shape)
#print(type(x))


    

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join
pathIn= 'output_images_subject2/'
pathOut = 'videotarget_subject2.avi'
fps = 5
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
#files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#files.sort(key = lambda x: x[5:-4])
for i in range(1000):
    filename=pathIn + files[i]
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()

In [ ]:
import os
if not os.path.isdir("epoch_images"):
    os.mkdir("epoch_images")

In [ ]:
v=x[3].reshape(256,512,3)
plt.imshow(v)

In [ ]:
import matplotlib.pyplot as plt
x=plt.imread("/kaggle/input/eveerybody-dance-nowsubject-1/subject1/train/train_label/frame000001.png")
x.shape

In [ ]:
g_model_json = g_model.to_json()
with open("g_modelx.json", "w") as json_file:
    json_file.write(g_model_json)
g_model.save_weights("modelgenx.h5")
print("Saved model to disk")
 

 


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from os import listdir
v=plt.imread('/kaggle/input/everybody-dance-now1/subject4/train/train_img/frame020001.png')
if not os.path.isdir("output_images"):
    os.mkdir("output_images")
x=plt.imshow(v)

In [ ]:
cv2.imwrite("output_images/1.png",v)

In [ ]:
#im_rgb = cv2.cvtColor(im_cv, cv2.COLOR_BGR2RGB)
x=plt.imread("output_images/1.png")
#im = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
plt.imshow(x)